In [1]:
# FOR NUMERICAL ANALYTICS
import numpy as np
# TO STORE AND PROCESS DATA IN DATAFRAME
import pandas as pd

# BASIC VISUALIZATION PACKAGE
import matplotlib.pyplot as plt
# ADVANCED PLOTING
import seaborn as seabornInstance
# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split
# STATS
from sklearn import datasets, metrics
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


ModuleNotFoundError: No module named 'sklearn'

In [15]:
#2015 data
df_15 = pd.read_csv('/home/akash/Desktop/predict.csv')
#df_15.describe()
#df_15.info()
usecols = ['Rank','Country','Score','GDP','Support',
                'Health','Freedom','Generosity','Corruption']

df_15.columns = ['Country','Rank','Score','Support',
                'GDP','Health',
                'Freedom','Generosity','Corruption']
df_15['Year'] = 2015 #add year column
df_15.head()

,Country,Rank,Score,Support,GDP,Health,Freedom,Generosity,Corruption,Year
0,Finland,Western Europe,7.8087,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,2015
1,Denmark,Western Europe,7.6456,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,2015
2,Switzerland,Western Europe,7.5599,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,2015
3,Iceland,Western Europe,7.5045,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,2015
4,Norway,Western Europe,7.4880,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,2015


In [16]:
target = ['Top','Top-Mid', 'Low-Mid', 'Low' ]
target_n = [4, 3, 2, 1]


In [17]:
evaluation = pd.DataFrame({'Model':[],
                          'Details':[],
                          'Root Mean Squared Error (RMSE)': [],
                          'R-squared (training)': [],
                          'Adjusted R-squared (training)': [],
                          'R-squared (test)':[],
                          'Adjusted R-squared(test)':[],
                           '5-Fold Cross Validation':[]
                          })

In [18]:
def adjustedR2(r2,n,k):
    return r2-(k-1)/(n-k)*(1-r2)

In [19]:
train_data, test_data = train_test_split(df_15, train_size = 0.8, random_state = 3)
lr = LinearRegression()
X_train = np.array(train_data['GDP'],
                   dtype = pd.Series).reshape(-1,1)
y_train = np.array(train_data['Score'], dtype = pd.Series)
lr.fit(X_train, y_train)

X_test = np.array(test_data['GDP'], 
                    dtype = pd.Series).reshape(-1,1)
y_test = np.array(test_data['Score'], dtype = pd.Series)

pred = lr.predict(X_test)
#ROOT MEAN SQUARED ERROR
rmsesm = float(format(np.sqrt(metrics.mean_squared_error(y_test,pred)),'.3f'))
#R-SQUARED (TRAINING)
rtrsm = float(format(lr.score(X_train, y_train),'.3f'))
#R-SQUARED (TEST)
rtesm = float(format(lr.score(X_test, y_test),'.3f'))
cv = float(format(cross_val_score(lr,df_15[['GDP']],df_15['Score'],cv=5).mean(),'.3f'))

print ("Average Score for Test Data: {:.3f}".format(y_test.mean()))
print('Intercept: {}'.format(lr.intercept_))
print('Coefficient: {}'.format(lr.coef_))
print('Happiness score = ',np.round(lr.intercept_,4),
      '+',np.round(lr.coef_,4),'* GDP')

r = evaluation.shape[0]
evaluation.loc[r] = ['Simple Linear Regression','-',rmsesm,rtrsm,'-',rtesm,'-',cv]
evaluation

Average Score for Test Data: 5.625
Intercept: 0.06287004011620123
Coefficient: [6.68447103]
Happiness score =  0.0629 + [6.6845] * GDP


,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared(test),5-Fold Cross Validation
0,Simple Linear Regression,-,0.599,0.56,-,0.678,-,-8.706


In [20]:
# MULTIPLE LINEAR REGRESSION 1
train_data_dm,test_data_dm = train_test_split(df_15,train_size = 0.8,random_state=3)
independent_var = ['GDP','Health','Freedom','Support','Generosity','Corruption']
complex_model_1 = LinearRegression()
complex_model_1.fit(train_data_dm[independent_var],train_data_dm['Score'])
print('Intercept: {}'.format(complex_model_1.intercept_))
print('Coefficients: {}'.format(complex_model_1.coef_))
print('Happiness score = ',np.round(complex_model_1.intercept_,4),
      '+',np.round(complex_model_1.coef_[0],4),'∗ Support',
      '+',np.round(complex_model_1.coef_[1],4),'* GDP', 
      '+',np.round(complex_model_1.coef_[2],4),'* Health',
      '+',np.round(complex_model_1.coef_[3],4),'* Freedom',
       '+',np.round(complex_model_1.coef_[4],4),'* Generosity',
      '+',np.round(complex_model_1.coef_[5],4),'* Corrption')
pred = complex_model_1.predict(test_data_dm[independent_var])
rmsecm = float(format(np.sqrt(metrics.mean_squared_error(
                       test_data_dm['Score'],pred)),'.3f'))
rtrcm = float(format(complex_model_1.score(
                        train_data_dm[independent_var],
                        train_data_dm['Score']),'.3f'))
artrcm = float(format(adjustedR2(complex_model_1.score(
                            train_data_dm[independent_var],
                            train_data_dm['Score']),
                            train_data_dm.shape[0],
                            len(independent_var)),'.3f'))
rtecm = float(format(complex_model_1.score(
                        test_data_dm[independent_var],
                        test_data_dm['Score']),'.3f'))
artecm = float(format(adjustedR2(complex_model_1.score(
                        test_data_dm[independent_var],test_data['Score']),
                        test_data_dm.shape[0],
                        len(independent_var)),'.3f'))
cv = float(format(cross_val_score(complex_model_1,
                    df_15[independent_var],
                    df_15['Score'],cv=5).mean(),'.3f'))
r = evaluation.shape[0]
evaluation.loc[r] = ['Multiple Linear Regression-1','selected features',rmsecm,rtrcm,artrcm,rtecm,artecm,cv]
evaluation.sort_values(by = '5-Fold Cross Validation', ascending=False)

Intercept: -2.196274809245388
Coefficients: [ 2.25965779  0.03688504  1.88560509  0.25395582  0.41399018 -0.51728365]
Happiness score =  -2.1963 + 2.2597 ∗ Support + 0.0369 * GDP + 1.8856 * Health + 0.254 * Freedom + 0.414 * Generosity + -0.5173 * Corrption


,Model,Details,Root Mean Squared Error (RMSE),R-squared (training),Adjusted R-squared (training),R-squared (test),Adjusted R-squared(test),5-Fold Cross Validation
1,Multiple Linear Regression-1,selected features,0.501,0.74,0.728,0.775,0.73,-6.224
0,Simple Linear Regression,-,0.599,0.56,-,0.678,-,-8.706


In [24]:
X = train_data_dm[independent_var]
y = train_data_dm['Score']
'''
 This function takes the features as input and
 returns the normalized values, the mean, as well 
 as the standard deviation for each feature. 
 '''
def featureNormalize(X):
    mu = np.mean(X) # Define the mean
    sigma = np.std(X) # Define the standard deviation.
    X_norm = (X - mu)/sigma # Scaling function.
    return X_norm, mu, sigma 

m = len(y) # length of the training data
X = np.hstack((np.ones([m,1]), X)) # Append the bias term (field containing all ones) to X.
y = np.array(y).reshape(-1,1) # reshape y to mx1 array
theta = np.zeros([7,1]) # Initialize theta (the coefficient) to a 3x1 zero vector.

''' 
 This function takes in the the values for 
 the training set as well as initial values 
 of theta and returns the cost(J).
 '''  

def cost_function(X,y, theta):
    h = X.dot(theta) # The hypothesis
    J = 1/(2*m)*(np.sum((h-y)**2)) # Implementing the cost function
    return J

In [25]:
num_iters = 2000 # Initialize the iteration parameter.
alpha = 0.01 # Initialize the learning rate.
def gradientDescentMulti(X, y, theta, alpha, iterations):
    m = len(y)
    J_history = []
    for _ in range(iterations):
        temp = np.dot(X, theta) - y
        temp = np.dot(X.T, temp)
        theta = theta - (alpha/m) * temp
        J_history.append(cost_function(X, y, theta)) # Append the cost to the J_history array
    return theta, J_history
theta , J_history = gradientDescentMulti(X, y, theta, alpha, num_iters)

<ipython-input-24-9a614c24f684>:27: RuntimeWarning: overflow encountered in square
  J = 1/(2*m)*(np.sum((h-y)**2)) # Implementing the cost function
<ipython-input-25-a7b3ffdd6f15>:9: RuntimeWarning: invalid value encountered in subtract
  theta = theta - (alpha/m) * temp
